## Sample game

In [1]:
import sys
import time
from copy import deepcopy
from IPython.display import clear_output

if '..' not in sys.path: sys.path.append('..')
from src.utils.common import *
from src.dnd.actions import *
from src.dnd.units import *
from src.dnd.game_utils import *
from src.dnd.game_board import DnDBoard
from src.agent.agent import DnDAgent
from src.agent.agent_utils import get_states, get_default_action_resolver

In [2]:
def create_game():
    game = DnDBoard((5, 5))
    ally = GenericSoldier("Ally soldier", attack_damage=25)
    ally2 = GenericSoldier("Ally ranger", attack_damage=25, health=50, range=4)
    enemy = GenericSoldier("Enemy soldier", attack_damage=25)
    enemy2 = GenericSoldier("Enemy ranger", attack_damage=25, health=50, range=4)

    place_unit_randomly(game, ally, 0)
    place_unit_randomly(game, enemy, 1)
    place_unit_randomly(game, ally2, 0)
    place_unit_randomly(game, enemy2, 1)

    game.initialize_game()
    action_resolver = get_default_action_resolver(game)

    color_map = {
        ally: "Green",
        enemy: "Red",
        ally2: "Blue",
        enemy2: "Purple"
    }

    return game, color_map, action_resolver

In [3]:
agent_path_1 = '../config-log/2v2-0/trained-agents/agent-gen4-10.2i'
agent_path_2 = '../config-log/2v2-0/trained-agents/agent-gen9-10.3i'
agent1 = DnDAgent.load_agent(agent_path_1)
agent1.epsilon = 0
agent2 = DnDAgent.load_agent(agent_path_2)
agent2.epsilon = 0

In [4]:
def play_loop(agent1, agent2, game: DnDBoard, color_map, random_action_resolver=None, manual_input=False, delay=0.5):
    game_over = False
    iter_count = 0

    print_game(game, color_map)
    while not game_over:
        try:
            unit, player_id = game.get_current_unit()

            iter_count += 1

            # sleeping 
            if manual_input:
                command = input()
                if command == 'stop':
                    raise KeyboardInterrupt()
                elif command == 'continue':
                    manual_input = False
            else:
                time.sleep(delay)
            
            clear_output(wait=True)
            print(f'Iteration: {iter_count}')

            agent = agent1 if player_id == 0 else agent2
            _, _, new_coords, action = get_states(game, agent, random_action_resolver)
            _, game_over = take_turn(game, new_coords, action, color_map, True)
        except KeyboardInterrupt:
            print(f'\nGame interrupted after {iter_count} iterations')
            return None

    winner = 0 if len(game.players_to_units[1]) else 1
    print(f'\nGame over in {iter_count} iterations. Winner: agent #{winner + 1}')

    return iter_count, winner

Start new game:

In [4]:
counter = []

In [9]:
game, color_map, action_resolver = create_game()

ic, win = play_loop(agent1, agent2, game, color_map, action_resolver, False, 1)

counter.append(win)

Iteration: 9
Turn made by player #0: `Ally ranger`:
	Unit moves: (3, 1) -> (3, 2);
	And takes aciton `Sword attack` with attributes: {'source_unit': 'Ally ranger', 'target_unit': 'Enemy soldier'}

Units alive: 2
Players: 2:
	Player #0 (2 units): `Ally soldier (25 HP)`, `Ally ranger (50 HP)`,   
	Player #1 (0 units):   
	
    0 1 2 3 4
 0            0 
 1            1 
 2     ██     2 
 3   ██       3 
 4            4 
    0 1 2 3 4

Next move is by player #0: `Ally ranger`

Game over in 9 iterations. Winner: agent #2


In [9]:
def play_loop_fast(agent1, agent2, game: DnDBoard, random_action_resolver=None, color_map=None):
    game_over = False
    iter_count = 0

    while not game_over:
        try:
            unit, player_id = game.get_current_unit()
            iter_count += 1

            agent = agent1 if player_id == 0 else agent2
            _, _, new_coords, action = get_states(game, agent, random_action_resolver)
            _, game_over = game.take_turn(new_coords, action, True)
        except KeyboardInterrupt:
            print(f'\nGame interrupted after {iter_count} iterations')
            return iter_count, -1

    winner = 0 if len(game.players_to_units[1]) else 1

    return iter_count, winner

In [10]:
from tqdm.auto import tqdm

for i in tqdm(range(100)):
    game, color_map, action_resolver = create_game()
    
    ic, win = play_loop_fast(agent1, agent2, game, action_resolver, color_map)
    
    counter.append(win)

  0%|          | 0/100 [00:00<?, ?it/s]

Turn made by player #0: `Ally soldier`
Turn made by player #0: `Ally ranger`
Turn made by player #1: `Enemy soldier`
Turn made by player #1: `Enemy ranger`
Turn made by player #0: `Ally soldier`
Turn made by player #0: `Ally ranger`
Turn made by player #1: `Enemy soldier`
Turn made by player #1: `Enemy ranger`
Turn made by player #0: `Ally ranger`
Turn made by player #1: `Enemy soldier`
----------------------------------------------------------------------------------------------------
Turn made by player #1: `Enemy soldier`
Turn made by player #0: `Ally soldier`
Turn made by player #1: `Enemy ranger`
Turn made by player #0: `Ally ranger`
Turn made by player #1: `Enemy soldier`
Turn made by player #0: `Ally soldier`
Turn made by player #1: `Enemy ranger`
Turn made by player #1: `Enemy soldier`
Turn made by player #0: `Ally soldier`
Turn made by player #1: `Enemy ranger`
Turn made by player #1: `Enemy soldier`
-----------------------------------------------------------------------------

In [30]:
npcounter = np.array(counter)
print('Agent 1 vs. Agent 2:')
print(f'{np.sum(npcounter == 0)} vs. {np.sum(npcounter == 1)}')

Agent 1 vs. Agent 2:
8236 vs. 7188
